In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns

In [2]:
df=pd.read_csv('train.csv')
df['date']=pd.to_datetime(dict(year=df.year,month=df.month,day=df.day))
date=df.pop('date')
df.insert(0,'date',date)
df.head()

,date,year,month,day,latitude,longitude,zon.winds,mer.winds,humidity,air temp.,s.s.temp.
0,1992-12-13,1992,12,13,-4.99,156.0,4.8,-2.8,78.5,28.98,30.06
1,1989-05-14,1989,5,14,0.00,-110.1,-2.5,2.4,NaN,24.95,25.23
2,1993-06-28,1993,6,28,2.04,-110.2,-4.5,5.9,NaN,25.78,27.84
3,1995-01-08,1995,1,8,1.98,-140.0,-7.7,1.2,82.6,26.47,26.86
4,1995-02-21,1995,2,21,5.01,-170.0,-5.4,-6.6,75.2,28.83,28.90


In [3]:
df.sort_values(['date'],inplace=True)
df.reset_index(inplace=True,drop=True)
#df.set_index('date',inplace=T)
df.dropna(subset=['air temp.'],inplace=True)
df['zon.winds'].fillna(np.random.normal(-3.505847,3.222261),inplace=True)
df['mer.winds'].fillna(np.random.normal(0.451069,2.927329),inplace=True)
df['humidity'].fillna(np.random.normal(81.267260,5.363381),inplace=True)

x=df.iloc[:,4:-1]
y=df.iloc[:,-1]
x.head()

,latitude,longitude,zon.winds,mer.winds,humidity,air temp.
0,-0.02,-109.44,-4.9,1.1,82.91374,25.66
1,-0.02,-109.44,-4.5,2.2,82.91374,25.69
2,-0.02,-109.44,-3.8,1.9,82.91374,25.56
3,-0.02,-109.44,-4.4,0.3,82.91374,24.72
4,-0.02,-109.44,-3.2,0.1,82.91374,24.66


In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error,root_mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, QuantileTransformer, KBinsDiscretizer, PolynomialFeatures, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit

tss=TimeSeriesSplit(n_splits=3)
for train,test in tss.split(x):
    x_train,x_test=x.iloc[train,:],x.iloc[test,:]
    y_train,y_test=y.iloc[train],y.iloc[test]

#x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20)
x_train.describe()

,latitude,longitude,zon.winds,mer.winds,humidity,air temp.
count,72429.000000,72429.000000,72429.000000,72429.000000,72429.000000,72429.000000
mean,0.496764,-52.714413,-2.997008,0.817622,82.138332,26.742258
std,4.457726,133.693907,3.224023,2.948548,3.894639,1.843959
min,-8.330000,-180.000000,-12.200000,-10.800000,54.000000,17.050000
25%,-2.020000,-140.100000,-5.500000,-1.200000,80.700000,25.840000
50%,0.010000,-110.100000,-3.500000,1.000000,82.913740,27.190000
75%,4.990000,147.000000,-0.434874,3.310107,82.913740,28.060000
max,9.030000,170.000000,12.500000,11.000000,99.600000,31.480000


In [ ]:
mod=KNeighborsRegressor()
mod.fit(X=x_train,y=y_train)
pred=mod.predict(x_test)
plt.scatter(pred,y_test)

In [ ]:
mean_absolute_error(y_test,pred)

In [ ]:
root_mean_squared_error(y_test,pred)

In [ ]:
mod=Pipeline([('scale',StandardScaler()),
              ('model',KNeighborsRegressor())])
#mod.get_params
mod.fit(X=x_train,y=y_train)
pred=mod.predict(x_test)
plt.scatter(pred,y_test)

In [ ]:
mean_absolute_error(y_test,pred)

In [ ]:
root_mean_squared_error(y_test,pred)

In [ ]:
mod=GridSearchCV(estimator=mod,
                 param_grid={'model__n_estimators': range(50,151)},
                 cv=3)
mod.fit(X=x_train,y=y_train)
pred=mod.predict(x_test)
plt.scatter(pred,y_test)

In [ ]:
mean_absolute_error(y_test,pred)

In [ ]:
root_mean_squared_error(y_test,pred)

In [ ]:
pd.DataFrame(mod.cv_results_)